# Habitica API Test Page
This page is to test queries against the Habitica API. I am primarily using it to inspect the structure of JSON data objects to help with mapping to C# classes.

To avoid authentication details being exposed to the world in this file, it requires the creation of a `.auth.cfg` (any name really, but that is the default) with the following content:

    [Habitica]
    url = https://habitica.com
    login = ;your user ID
    password = ;your API key

Note that the section name can be anything, and you can use multiple sections for multiple accounts.

# Setup

In [2]:
import os
import requests
from configparser import ConfigParser

In [3]:
class ConfigError(Exception):
    def __init__(self, value):
        self.value = value
        
    def __str__(self):
        return repr(self.value)

In [4]:
def load_auth(config_file='~/.auth.cfg', section='Habitica'):
    headers = {}
    config_file = os.path.expanduser(config_file)
    
    with open(config_file) as cf:
        config = ConfigParser()
        config.read_file(cf)

        try:
            headers = {'url': 'https://habitica.com',
                  'x-api-user': config.get(section, 'userid'),
                  'x-api-key': config.get(section, 'apikey')}
        except configparser.NoSectionError:
            raise ConfigError("No '%s' section in '%s'" % section, configfile)
        except configparser.NoOptionError as e:
            raise ConfigError("Missing option in auth file '%s': %s" % (configfile, e.message))

    return headers

In [5]:
headers = load_auth(section='Habitica_testuser')
#headers = load_auth(section='Habitica_deecee')
headers['url'] += '/api/v3/'

# HTTP Request Helpers

In [6]:
def get(command):
    r = requests.get(headers['url']+command, headers=headers)
    return r.json()

# Status

In [7]:
get('status')

{'data': {'status': 'up'}, 'success': True}

# User

In [8]:
user = get('user')['data']

In [9]:
user.keys()

dict_keys(['purchased', 'items', 'notifications', 'tasksOrder', 'newMessages', 'guilds', 'balance', 'profile', 'migration', 'id', 'preferences', 'challenges', 'lastCron', 'achievements', 'inbox', 'extra', 'tags', 'flags', '_v', 'pushDevices', '_id', 'invitations', 'party', 'stats', 'backer', 'auth', 'history', 'contributor'])

In [10]:
user['history']

{'exp': [{'date': '2016-07-25T08:06:50.474Z', 'value': 151},
  {'date': '2016-07-26T04:11:30.331Z', 'value': 478},
  {'date': '2016-07-27T06:41:34.368Z', 'value': 497},
  {'date': '2016-07-28T05:03:45.757Z', 'value': 497},
  {'date': '2016-07-29T14:29:03.132Z', 'value': 497}],
 'todos': [{'date': '2016-07-25T08:06:50.474Z', 'value': 0},
  {'date': '2016-07-26T04:11:30.331Z', 'value': 0},
  {'date': '2016-07-27T06:41:34.368Z', 'value': -2},
  {'date': '2016-07-28T05:03:45.757Z', 'value': -4.05191340925413},
  {'date': '2016-07-29T14:29:03.132Z', 'value': -1}]}

In [11]:
user['tags']

[{'id': '31a73ce4-06ac-4102-9ffa-e60aa3dfd7c3', 'name': 'morning'},
 {'id': '7fa966dd-5990-44b3-8764-d528d569ca0e', 'name': 'afternoon'},
 {'id': '0821e53c-dd4c-4256-8b7e-2f8919ab7ffc', 'name': 'evening'}]

# Member

In [12]:
member = get('members/'+headers['x-api-user'])['data']

In [13]:
member.keys()

dict_keys(['auth', 'party', 'profile', 'preferences', 'id', 'items', 'achievements', '_id', 'stats'])

In [14]:
member

{'_id': '90707bd8-7dd8-4da6-880c-d57eb7814d3c',
 'achievements': {'challenges': [],
  'perfect': 4,
  'ultimateGearSets': {'healer': False,
   'rogue': False,
   'warrior': False,
   'wizard': False}},
 'auth': {'timestamps': {'created': '2016-07-16T12:51:07.433Z',
   'loggedin': '2016-07-29T14:29:03.132Z'}},
 'id': '90707bd8-7dd8-4da6-880c-d57eb7814d3c',
 'items': {'eggs': {'Wolf': 1},
  'food': {'CottonCandyBlue': 2, 'CottonCandyPink': 1, 'Meat': 1},
  'gear': {'costume': {'armor': 'armor_base_0',
    'head': 'head_base_0',
    'shield': 'shield_base_0'},
   'equipped': {'armor': 'armor_base_0',
    'eyewear': 'eyewear_special_blackTopFrame',
    'head': 'head_base_0',
    'shield': 'shield_base_0'},
   'owned': {'eyewear_special_blackTopFrame': True,
    'eyewear_special_blueTopFrame': True,
    'eyewear_special_greenTopFrame': True,
    'eyewear_special_pinkTopFrame': True,
    'eyewear_special_redTopFrame': True,
    'eyewear_special_whiteTopFrame': True,
    'eyewear_special_yell

# Task

### Find the common set of keys to all task types

In [15]:
all_tasks = get('tasks/user')['data']

In [16]:
habits = [task for task in all_tasks if task['type'] == 'habit']
dailies = [task for task in all_tasks if task['type'] == 'daily']
todos = [task for task in all_tasks if task['type'] == 'todo']
rewards = [task for task in all_tasks if task['type'] == 'reward']

In [17]:
with_alias = [t['type'] for t in all_tasks if 'alias' in t]
with_alias

['habit', 'daily', 'todo', 'daily']

In [18]:
tasks = [
    set(habits[0]),
    set(rewards[0]), 
    set(todos[0]), 
    set(dailies[0])]

base_task = tasks[0]
for t in tasks[1:]:
    base_task = base_task.intersection(t)
    
base_task

{'_id',
 'attribute',
 'challenge',
 'createdAt',
 'id',
 'notes',
 'priority',
 'reminders',
 'tags',
 'text',
 'type',
 'updatedAt',
 'userId',
 'value'}

In [19]:
# all keys in habit that are not in base_task
tasks[0].difference(base_task)

{'alias', 'down', 'history', 'up'}

In [20]:
# all keys in reward that are not in base_task
tasks[1].difference(base_task)

set()

In [21]:
# all keys in todo that are not in base_task
tasks[2].difference(base_task)

{'alias', 'checklist', 'collapseChecklist', 'completed'}

In [22]:
# all keys in daily that are not in base_task
tasks[3].difference(base_task)

{'alias',
 'checklist',
 'collapseChecklist',
 'completed',
 'everyX',
 'frequency',
 'history',
 'repeat',
 'startDate',
 'streak'}

### Examine components of a task in detail

In [23]:
dailies[1]

{'_id': 'a3c0a44b-4f13-4601-a407-f151f93e04a0',
 'alias': 'ddd',
 'attribute': 'str',
 'challenge': {},
 'checklist': [],
 'collapseChecklist': False,
 'completed': False,
 'createdAt': '2016-07-28T05:07:26.885Z',
 'everyX': 3,
 'frequency': 'daily',
 'history': [{'date': 1469802543155, 'value': -1}],
 'id': 'a3c0a44b-4f13-4601-a407-f151f93e04a0',
 'notes': 'this is some extra notes',
 'priority': 0.1,
 'reminders': [],
 'repeat': {'f': True,
  'm': True,
  's': True,
  'su': True,
  't': True,
  'th': True,
  'w': True},
 'startDate': '2016-07-27T16:00:00.000Z',
 'streak': 0,
 'tags': [],
 'text': 'repeat every 3 days',
 'type': 'daily',
 'updatedAt': '2016-07-29T14:29:03.299Z',
 'userId': '90707bd8-7dd8-4da6-880c-d57eb7814d3c',
 'value': -1}

# Groups

In [24]:
groupId = 'habitrpg'
g = get('groups/' + groupId)
g

{'data': {'_id': '00000000-0000-4000-A000-000000000000',
  'balance': 0,
  'challengeCount': 1158,
  'chat': [{'backer': {},
    'contributor': {'admin': False,
     'contributions': 'Dutch translation',
     'level': 1,
     'text': 'Linguist'},
    'flagCount': 0,
    'flags': {},
    'id': '4a94b403-457e-40c9-9b1f-334ebcd4597d',
    'likes': {},
    'text': '@ChronosXIII  If you search for a party, try the [Party wanted (Looking for group)](https://habitica.com/#/options/groups/guilds/f2db2a7f-13c5-454d-b3ee-ea1f5089e601) guild and leave a message if you need a party or party members. There is no real etiquette; some parties may search for players with a specific class, others may only want players who live in the same timezone or close enough so that the buff skills lasts longer for everyone, and others just want people',
    'timestamp': 1469801759806,
    'user': 'Dracindo',
    'uuid': '1ee37752-f615-45a2-872e-8bc4eff2255b'},
   {'backer': {},
    'contributor': {},
    'flagCou